In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta

In [3]:
import devanalyst.simulation.statics as S_
from devanalyst.simulation.businessObjects import UserStory, UserStoriesRepo, Ticket, TicketsRepo, WorkItem, \
UserStoryStatus, Backlog, ScrumTeam, ScrumTeamsRepo, WorkAssignments, ReleaseCycleContext
import devanalyst.simulation.simulationModels as simm

importing Jupyter notebook from c:\users\aleja\documents\code\chateauclaudia-labs\devanalyst\devanalyst\simulation\statics.ipynb
importing Jupyter notebook from c:\users\aleja\documents\code\chateauclaudia-labs\devanalyst\devanalyst\simulation\businessObjects.ipynb
importing Jupyter notebook from c:\users\aleja\documents\code\chateauclaudia-labs\devanalyst\devanalyst\simulation\simulationModels.ipynb


<h1>User Stories</h1>
<p>Capacity planning for the release: we create a backlog of stories. Initial resource allocations are made, but they might change in a sprint if needed</p>

In [4]:
class IdCounter:
    def __init__(self):
        self.counter = 1
        
    def increment(self):
        self.counter += 1
        
    def next_id(self):
        return self.counter

In [5]:
def generateNextUserStory(nextId, team, bandwidth, sprintDuration, modelsConfig):
# Returns a UserStory, randomly choosing the amount of effort the UserStory might take (while fitting withing a sprint),
# and assigning to a random developer in the team with enough time to do it, to a spec written by a randomly chosen 
# product manager from the team.
#
# Random choices are made using the random generator in the modelsConfig parameter.
#
# Depletes the time this UserStory would take from the bandwidth for the developer in question.
#
# If no developer in the team has time to do such a UserStory, returns 'None'

    estimate = modelsConfig.random.pickHowLong(sprintDuration)

    available = bandwidth[bandwidth >= estimate] # Subset of developers with enough time to do this user story
    if (len(available) == 0):
        return None
    
    developer = modelsConfig.random.pickOneIdx(available)
    bandwidth[developer] -= estimate # deplete capacity now earmarked for this user story

    productManager = modelsConfig.random.pickOne(team.productManagers)
    
    return UserStory('UserStory #' + str(nextId), estimate, developer, productManager)

In [6]:
def canResourceMoreWork(bandwidth, sprintDuration):
# Helper function used in createUserStoryBacklog.
# Returns a boolen on whether the team has enough capacity left in 'bandwidth' to resource one more user story
#
# -bandwidth: Pandas Series indexed on developers' names, with the values being the amount of unallocated days
# for that developer.
# -sprintDuration: integer representing duration of a sprint, in number of days 

    # If over half the developers in the team have at least half a sprint left, still can resource more work
    numberOfDevelopers = bandwidth.size
    condition1 = bandwidth[bandwidth > sprintDuration/2.0].size > numberOfDevelopers / 2.0
    
    # If any developer in the team has more than 150% of a sprint left, then can still resource more work
    condition2 = bandwidth.max() >= 1.5 * sprintDuration
    
    return condition1 or condition2

In [7]:
def createUserStoryBacklog(team, releaseDuration, sprintDuration, counter, modelsConfig):
#Creates user stories that a scrum team should work on for a release. Algorithm basically determines how much time 
# there is in a planned duration for a release, and based on that randomly creates user stories associated with the
# areas of responsibility of the scrum team in question, until capacity is filled to between 90% and 100%. Each user story 
# is supposed to be completed in a sprint, so a user story's estimated duration is ramdomly generated to be between
# 10% and 100% of the sprint's duration period.
#
# -team: the ScrumTeam for which the backlog is being created
# -releaseDuration: integer number of business days to the intended release date. Defaults to around 6 months
# -sprintDuration: integer number of business days that a sprint lasts. Defaults to 2 weeks. 
# -counter: an IdCounter, to tell us what is the id for the next user story to create. This method increments the counter
# -modelsConfig: a ModelsConfig instance with a random generator used in the creation of user stories.
# with each user story that is created.

    numberOfDevelopers = len(team.developers)
    initialCapacity = numberOfDevelopers *releaseDuration # number of work-days available for the release
    bandwidth = pd.Series(releaseDuration, team.developers) # initialize bandwidth to all the time left in the release

    #numberOfSprints = releaseDuration/sprintDuration
    stories = []
    backlog = Backlog()     
    
    # We allocate as much of the team as practical, with user stories ranging in estimated duration from 1 day
    # to the number of days in a sprint. To avoid artificial situations, don't aim to fill every single day,
    # so will stop when we have allocated at least 10% and there is no developer left with enough bandwidth
    # to resource a medium-size user story.
    # To avoid infinite loops, we force a stop after 1000 tries, though most likely we will exit well before then.
    next_id = counter.next_id()
    while (canResourceMoreWork(bandwidth, sprintDuration) and next_id < 10000): #To avoid infinite loops, cap number of user stories
        story = generateNextUserStory(next_id, team, bandwidth, sprintDuration, modelsConfig) #this call mutates bandwidth
        if (story == None):
            continue #Try again, maybe we tried for a very large estimate but a smaller user story can still be resourced
        stories.append(story)
        
        counter.increment()
        next_id = counter.next_id()
        backlog.pendingUserStories.append(UserStoryStatus(story.userStoryId))       
    
    return stories, backlog

<h1>Do a sprint</h1>

In [8]:
def chooseWhatToDoInSprint(modelsConfig):
# At the start of a sprint, elects what items a team will do in a sprint
#
# -teamId: string identifier of team for which we are choosing work for this sprint
# sprintDuration: length of the sprint for which work needs to be chosen, in man-days
# -modelsConfig: a ModelsConfig instance with parameters for making the choice of work to do in the sprint
# -sprint: integer representing which sprint we are in

    
    allocationModel = modelsConfig.allocationModel
    work = WorkAssignments(modelsConfig.context, modelsConfig.globalRepo)
    
    return allocationModel.allocate(work, modelsConfig) # Mutates work

In [9]:
def inflowOfTickets(modelsConfig):
# Returns an array of newly generated tickets against the work done by the team
    bugs = []
    for qualityModel in modelsConfig.qualityModels:
        bugs.extend(qualityModel.findBugs(modelsConfig))
    return bugs

In [10]:
def bookDeveloperEffort(items, budget, modelsConfig):
# Helper class, used as part of the book keeping involved in delivering a sprint.
#
# Returns what portion of the budget, if any, was left over after booking the work done by the developer
#
# -items: a list of WorkItem objects that record work that should be done in a sprint by a single developer. 
# It will be mutated by this fundtion by recording what were the actual number of man-days spent on each WorkItem, and to what degree it was completed
# -budget: number of man-days that the developer was given to try to complete the WorkItems
# -modelsConfig: a ModelsConfig instance to model how real costs differ from estimates

    for task in items:
        if budget <= 0:
            break # No more tasks progressed during this sprint
        realCost = simm.computeRealCost(task, modelsConfig)
        if realCost == 0:
            continue # Boundary base. Shouldn't happen, but if someone entered an estimate of 0 don't want to divide by 0
        if realCost <= budget:
            task.percentAchieved = 1
            task.actual = realCost
            budget -= realCost
        else:
            # Can only complete part of the item
            task.percentAchieved = budget/realCost
            task.actual = budget
            budget = 0
    return budget

In [11]:
def deliverSprint(work, modelsConfig):
# Based on the team's productivy during the sprint, record how much of the work initially planned for the sprint
# actually gets done
#
# -work: a WorkAssignment as it was at the start of the sprint. It is mutated by this method by recording, for each
# WorkItem in the WorkAssignment, what percentage of it got accomplished
# -sprintDuration: number of man-days that sprint lasted
# -modelsConfig: a ModelsConfig instance to model how real costs differ from estimates

    teamId                 = modelsConfig.context.teamId
    sprintDuration         = modelsConfig.context.sprintDuration
    sprint                 = modelsConfig.context.sprint

    teamsRepo              = modelsConfig.globalRepo.teamsRepo
    storiesRepo            = modelsConfig.globalRepo.storiesRepo
    ticketsRepo            = modelsConfig.globalRepo.ticketsRepo

    team = teamsRepo.findTeam(teamId)
    for person in team.developers:
        budget = sprintDuration # Developer has up to these many days to complete work in the sprint
        
        # First, deliver the work from the current sprint
        budget = bookDeveloperEffort(work.getWorkItems(person, S_.CURRENT_SPRINT), budget, modelsConfig)
                
        # Second, developer still has some time in his/her budget for this sprint, then he/she might have started to work
        # on things for the next sprint, in which case record that progress
        budget = bookDeveloperEffort(work.getWorkItems(person, S_.NEXT_SPRINT), budget, modelsConfig)    

In [12]:
def updateBacklogAfterSprint(work, modelsConfig):
# At the end of a sprint, updates the backlog of a team based on the work completed in this sprint
#
# -work: a WorkAssignment, which reflects what was accomplished during the sprint through the field percentAchieved
# in each of the WorkItems in the various arrays within the work.allocations dictionary

    teamId                 = modelsConfig.context.teamId
    teamsRepo              = modelsConfig.globalRepo.teamsRepo

    team = teamsRepo.findTeam(teamId)
    completedWork = []
    for person in team.developers:
        completedWork.extend(work.getWorkItems(person, S_.CURRENT_SPRINT))
        completedWork.extend(work.getWorkItems(person, S_.NEXT_SPRINT))
    
    team.backlog.updateStatus(completedWork, modelsConfig.context, modelsConfig.globalRepo) 

<h1>Functions to Generate Timecards</h1>

In [13]:
def addBusinessDays(start, duration):
# Returns a datetime.date object which is several business days after the input 'start'
    remainsToAdd = duration
    current = start
    while remainsToAdd > 0:
        current = current + timedelta(1)
        weekday = current.weekday()
        if weekday >= 5: # sunday = 6
            continue
        remainsToAdd -= 1
    return current

def subtractBusinessDays(start, duration):
# Returns a datetime.date object which is several business days before the input 'start'
    remainsToSubtract = duration
    current = start
    while remainsToSubtract > 0:
        current = current - timedelta(1)
        weekday = current.weekday()
        if weekday >= 5: # sunday = 6
            continue
        remainsToSubtract -= 1
    return current

In [17]:
# Maintains a log of the entire release cycle. It contains nested dictionaries that culminate in a Pandas dataframe.
class ReleaseLog:
# The hierarchy of nested dictionaries is as follows:
#
# -The root dictionary is self.log
# -The next 3 levels break out depending on teamId, sprint, and 'snapshot type', where 'snapshot type' is a sting
#  describing the kind of data being 'snapshot-ed' in a Pandas dataframe.
#  For example, these are all Pandas dataframes:
#      -self.log['Team A'][5]['Outcome']:                       outcome of work done by team A in sprint 5.
#      -self.log['Team B'][12]['planned_Start_CURRENT_SPRINT']: count of user stories planned for a sprint, where the
#                                                               counts are per each user story size and each level 
#                                                               of progress, as of the start of sprint 12 for team B
#                                                               done by team A in sprint 5.
#      -self.log['Team B'][12]['planned_End_CURRENT_SPRINT']:   Like the previous one, but as of the end of the sprint 
#                                                               so it reflects what was accomplished in the sprint.
#      -self.log['Team B'][12]['planned_Start_NEXT_SPRINT']   
#       & self.log['Team B'][12]['planned_End_NEXT_SPRINT']:    Like the previous two, but for 'bonus' work that is for 
#                                                               the next sprint but might be started in sprint 12 if 
#                                                               people finish sprint 12 tasks early.
#      -self.log['Team B'][12]['backlog']:                      Like the previous four, but for work that is not yet
#                                                               'planned', i.e., not earmarked for either the
#                                                               CURRENT_SPRINT (12) or NEXT_SPRINT (13)
#      -self.log['Team A'][5]['Resourcing']:                    Indicates resourcing allocations per developer for the
#                                                               tasks planned by team A in sprint 5. It is an aggregated
#                                                               result, i.e., indicates number of person-days for
#                                                               user story implementation or bug fixing, but does not
#                                                               list the specific user stories or bugs.
#
    BREAKOUT = 'Breakout'
    COMPLETED = 'Completed'
    REGRESSED = 'Regressed' # Used for user stories considered "completed" but which have open Tickets against them
    NEW_WORK = 'New Work'
    NOT_STARTED = 'Not Started'
    OVER_BUDGET = 'Over Budget'
    PRIOR_PROGRESSED = 'Prior Progressed'
    PRIOR_TO_FINISH = 'Prior to Finish'
    PROGRESSED = 'Progressed'
    TO_FINISH = 'To Finish'
    UNPLANNED = 'Unplanned'
    
    SNAPSHOTS = ['planned_Start_CURRENT_SPRINT', 'planned_End_CURRENT_SPRINT', 
                      'planned_Start_NEXT_SPRINT', 'planned_End_NEXT_SPRINT', 
                      'backlog', 'Resourcing', 'Outcome']  
    
    def __init__(self):
        self.log = {}
        
    
    def _getPlans(self, work):
        unassigned = work.allocations[S_.UNPLANNED][S_.OWNER_TBD]
        unplanned = []
        unplanned.extend(unassigned[S_.PRODUCTION_BUGS])
        unplanned.extend(unassigned[S_.DEV_TIME_BUGS])
        unplanned.extend(unassigned[S_.UNFINISHED_STORIES])
    
        planned_CURRENT_SPRINT = []
        subwork = work.allocations[S_.CURRENT_SPRINT]
        for person in subwork.keys():
            planned_CURRENT_SPRINT.extend(subwork[person][S_.PRODUCTION_BUGS])
            planned_CURRENT_SPRINT.extend(subwork[person][S_.DEV_TIME_BUGS])
            planned_CURRENT_SPRINT.extend(subwork[person][S_.UNFINISHED_STORIES])

        planned_NEXT_SPRINT = []
        subwork = work.allocations[S_.NEXT_SPRINT]
        for person in subwork.keys():
            planned_NEXT_SPRINT.extend(subwork[person][S_.PRODUCTION_BUGS])
            planned_NEXT_SPRINT.extend(subwork[person][S_.DEV_TIME_BUGS])
            planned_NEXT_SPRINT.extend(subwork[person][S_.UNFINISHED_STORIES])
            
        return planned_CURRENT_SPRINT, planned_NEXT_SPRINT, unplanned
        
    def snapshotSprintBacklog(self, modelsConfig):

        teamId                 = modelsConfig.context.teamId
        sprint                 = modelsConfig.context.sprint

        if (teamId not in self.log.keys()):
            self.log[teamId] = {}
        if (sprint not in self.log[teamId].keys()):
            self.log[teamId][sprint] = {}
    
        self.log[teamId][sprint]['backlog']             = ReleaseLog._build_backlog_df(modelsConfig.context,
                                                                                       modelsConfig.globalRepo)
    def snapshotSprintPlan(self, work, modelsConfig):

        teamId                 = modelsConfig.context.teamId
        sprintDuration         = modelsConfig.context.sprintDuration
        sprint                 = modelsConfig.context.sprint

        teamsRepo              = modelsConfig.globalRepo.teamsRepo
        storiesRepo            = modelsConfig.globalRepo.storiesRepo
        ticketsRepo            = modelsConfig.globalRepo.ticketsRepo

        if (teamId not in self.log.keys()):
            self.log[teamId] = {}
        if (sprint not in self.log[teamId].keys()):
            self.log[teamId][sprint] = {}
    
        planned_CURRENT_SPRINT, planned_NEXT_SPRINT, unplanned = self._getPlans(work)
        
        phase = 'Start'
        self.log[teamId][sprint]['planned_' + phase + '_CURRENT_SPRINT'] = ReleaseLog._build_plan_df(planned_CURRENT_SPRINT,
                                                                                                     modelsConfig.context,
                                                                                                    modelsConfig.globalRepo)
        self.log[teamId][sprint]['planned_' + phase + '_NEXT_SPRINT']    = ReleaseLog._build_plan_df(planned_NEXT_SPRINT,
                                                                                                      modelsConfig.context,
                                                                                                    modelsConfig.globalRepo)
        self.log[teamId][sprint]['Resourcing'] = work.committedTime(modelsConfig.context.sprintDuration)

        
    def snapshotSprintOutcome(self, work, modelsConfig):

        teamId                 = modelsConfig.context.teamId        
        sprintDuration         = modelsConfig.context.sprintDuration
        sprint                 = modelsConfig.context.sprint

        teamsRepo              = modelsConfig.globalRepo.teamsRepo
        storiesRepo            = modelsConfig.globalRepo.storiesRepo
        ticketsRepo            = modelsConfig.globalRepo.ticketsRepo

        if (teamId not in self.log.keys()):
            self.log[teamId] = {}
        if (sprint not in self.log[teamId].keys()):
            self.log[teamId][sprint] = {}
    
        planned_CURRENT_SPRINT, planned_NEXT_SPRINT, unplanned = self._getPlans(work)
        
        phase = 'End'
        self.log[teamId][sprint]['planned_' + phase + '_CURRENT_SPRINT'] = ReleaseLog._build_outcome_df(planned_CURRENT_SPRINT,
                                                                                                        modelsConfig.context,
                                                                                                        modelsConfig.globalRepo)
        self.log[teamId][sprint]['planned_' + phase + '_NEXT_SPRINT']    = ReleaseLog._build_outcome_df(planned_NEXT_SPRINT,
                                                                                                     modelsConfig.context,
                                                                                                        modelsConfig.globalRepo)
        self.log[teamId][sprint]['Outcome'] = work.committedTasks()
        
    def _getCategory(estimate):
        cat = int(estimate)
        return cat
        
    def _build_plan_df(allocations, context, globalRepo):

        sprintDuration         = context.sprintDuration

        teamsRepo              = globalRepo.teamsRepo
        storiesRepo            = globalRepo.storiesRepo
        ticketsRepo            = globalRepo.ticketsRepo

        cols = [ReleaseLog.BREAKOUT]
        categories = []
        # Used categories go 1, 2, ..., sprintDuration, corresponding to the original estimates for user stories. 
        # Spurious category 0 will be never used, but no harm in having it.
        for x in range(sprintDuration + 1): 
            
            category = x
            categories.append(category)
        cols.extend(categories)

        prior_progressed = [ReleaseLog.PRIOR_PROGRESSED]
        prior_remaining = [ReleaseLog.PRIOR_TO_FINISH]
        new_work = [ReleaseLog.NEW_WORK] 
        regressed = [ReleaseLog.REGRESSED]

        for cat in categories:
            prior_progressed.append(0.0)
            prior_remaining.append(0.0)
            new_work.append(0.0)
            regressed.append(0.0)
        
        bugs_per_story = {} # Dictionary where keys are user story ids and values are the bugs associated with it
        for item in allocations:
            story = storiesRepo.findStory(item.userStoryId)
            uss = teamsRepo.getUserStoryStatus(item.userStoryId)
            cat = ReleaseLog._getCategory(story.originalEstimate)
            idx = categories.index(cat) + 1 # offset by 1 since first element in arrays is the layer
        
            if (uss.percentAchieved == 0.0):
                new_work[idx] += 1
            else:
                if (uss.percentAchieved < 1.0):
                    prior_progressed[idx] += (story.originalEstimate * uss.percentAchieved)/story.originalEstimate
                    prior_remaining[idx] += (story.originalEstimate * (1-uss.percentAchieved))/story.originalEstimate
                else: 
                # Must be a bug on a story already completed. So we need to increment the 'regressed' counter, but this
                # is tricky: even if there
                # are multiple bugs on a user story we want to increment it only once. Therefore we do so only if the 
                # 'bugs_per_story' records no prior bugs for this story in this loop, i.e. for the first bug
                    ticket = ticketsRepo.findTicket(item.ticketId)
                    assert(ticket != None)
                    if item.userStoryId not in bugs_per_story.keys(): # First bug we see on this user story
                        regressed[idx] += 1
                        bugs_per_story[item.userStoryId] = [ticket]
                    else:
                        bugs_per_story[item.userStoryId].append(ticket)

                    
                    
                    
         
        data = []
        data.append(prior_progressed)
        data.append(prior_remaining)
        data.append(new_work)
        data.append(regressed)
        data_df = pd.DataFrame(data, columns=cols)
        return data_df
    
    def _build_outcome_df(allocations, context, globalRepo):

        sprintDuration         = context.sprintDuration

        teamsRepo              = globalRepo.teamsRepo
        storiesRepo            = globalRepo.storiesRepo
        ticketsRepo            = globalRepo.ticketsRepo
        
        cols = [ReleaseLog.BREAKOUT]
        categories = []
        for x in range(sprintDuration + 1): # inlude 0 as a category for tasks with less than 1 person-day to go
            category = x
            categories.append(category)
        cols.extend(categories)

        completed = [ReleaseLog.COMPLETED]
        progressed = [ReleaseLog.PROGRESSED]
        to_finish = [ReleaseLog.TO_FINISH] 
        not_started = [ReleaseLog.NOT_STARTED] 
        over_budget = [ReleaseLog.OVER_BUDGET] 
        regressed = [ReleaseLog.REGRESSED]

        for cat in categories:
            completed.append(0.0)
            progressed.append(0.0)
            to_finish.append(0.0)
            not_started.append(0.0)
            over_budget.append(0.0)
            regressed.append(0.0)
        
        # Dictionary where keys are user story ids and values are two dictionaries: one for open bugs and one for
        # closed bugs associated with it
        bugs_per_story = {} 
        OPEN_BUGS    = 'OPEN_BUGS'
        CLOSED_BUGS  = 'CLOSED_BUGS'
        
        for item in allocations:
            story = storiesRepo.findStory(item.userStoryId)
            uss = teamsRepo.getUserStoryStatus(item.userStoryId)
            cat = ReleaseLog._getCategory(story.originalEstimate)
            idx = categories.index(cat) + 1 # offset by 1 since first element in arrays is the layer
        
            if (uss.percentAchieved == 1.0):
                # Two possibilities: either we 1) completed the story in this sprint, or 2) completed it in a prior
                # sprint and this WorkItem is for a bug on that story. In case 1) we increment the 'completed' counter,
                # and in case 2) we need to increment the 'regressed' counter, but the latter is tricky: even if there
                # are multiple bugs on a user story we want to increment it only once. Therefore we do so only if the 
                # 'bugs_per_story' records no prior bugs for this story in this loop, i.e., for the first bug
                if (item.taskType == S_.UNFINISHED_STORIES):
                    completed[idx] += 1
                else: # Must be a bug
                    ticket = ticketsRepo.findTicket(item.ticketId)
                    assert(ticket != None)
                    # Record this is a  ticket. After loop in another loop we increment counters
                    if item.userStoryId not in bugs_per_story.keys(): #Create the key with an empty value
                        bugs_per_story[item.userStoryId] = {OPEN_BUGS: [], CLOSED_BUGS: []}
                        
                    if item.percentAchieved == 1.0:  
                        bugs_per_story[item.userStoryId][CLOSED_BUGS].append(ticket)
                    else:
                        bugs_per_story[item.userStoryId][OPEN_BUGS].append(ticket)
            else:
                if (uss.percentAchieved == 0.0):
                    not_started[idx] += 1
                else:
                    progressed[idx] += (story.originalEstimate * uss.percentAchieved)/story.originalEstimate
                    to_finish[idx] += (story.originalEstimate * (1-uss.percentAchieved))/story.originalEstimate
                    
            over_budget[idx] += (item.actual - item.estimate*item.percentAchieved)/story.originalEstimate
            
        # We need to loop again, not over the items but over the bugs we detected. Bugs are always against stories
        # completed in prior releases. So the counters to increment are associated to regressions for the story, and
        # even if there are multiple bugs we only increment one counter and only once (since we are counting 
        # stories in the logs, not bugs). So if there are open bugs, count it as a regression. Otherwise if
        # there are no open bugs but there are closed bugs, count it as a completion. Else no counter grows            
        for storyId in bugs_per_story.keys():
            story = storiesRepo.findStory(storyId)
            cat = ReleaseLog._getCategory(story.originalEstimate)
            idx = categories.index(cat) + 1 # offset by 1 since first element in arrays is the layer
            if len(bugs_per_story[storyId][OPEN_BUGS]) > 0: # we didn't fix all the bugs we set out to in the sprint
                regressed[idx] += 1
            else:
                if len(bugs_per_story[storyId][CLOSED_BUGS]) > 0: # we planned to fix bugs and we fixed them all
                    completed[idx] += 1
         
        data = []
        data.append(completed)
        data.append(progressed)
        data.append(to_finish)
        data.append(not_started)
        data.append(over_budget)
        data.append(regressed)
        data_df = pd.DataFrame(data, columns=cols)
        return data_df

    def _build_backlog_df(context, globalRepo):
        
        teamId                 = context.teamId
        sprintDuration         = context.sprintDuration

        teamsRepo              = globalRepo.teamsRepo
        storiesRepo            = globalRepo.storiesRepo
        ticketsRepo            = globalRepo.ticketsRepo
        
        team = teamsRepo.findTeam(teamId)
        backlog = team.backlog.pendingUserStories
        
        cols = [ReleaseLog.BREAKOUT]
        categories = []
        for x in range(sprintDuration + 1): # inlude 0 as a category for tasks with less than 1 person-day to go
            category = x
            categories.append(category)
        cols.extend(categories)

        unplanned = [ReleaseLog.UNPLANNED]
        completed = [ReleaseLog.COMPLETED]
        planned_progressed = [ReleaseLog.PROGRESSED]
        planned_remaining = [ReleaseLog.TO_FINISH]
        not_started = [ReleaseLog.NOT_STARTED]
        regressed = [ReleaseLog.REGRESSED]

        for cat in categories:
            unplanned.append(0.0)
            completed.append(0.0)
            planned_progressed.append(0.0)
            planned_remaining.append(0.0)
            not_started.append(0.0)
            regressed.append(0.0)
        
        for uss in backlog:
            story = storiesRepo.findStory(uss.userStoryId)
            cat = ReleaseLog._getCategory(story.originalEstimate)
            idx = categories.index(cat) + 1 # offset by 1 since first element in arrays is the layer
        
            if (uss.percentAchieved == 0.0):
                if (uss.planned == False):
                    unplanned[idx] += 1
                else:
                    not_started[idx] += 1
            else:
                if (uss.percentAchieved == 1.0):
                    bugs = ticketsRepo.getOpenTickets(uss.userStoryId)
                    if (len(bugs) > 0):
                        regressed[idx] += 1
                    else:
                        completed[idx] += 1                    
                else:
                    planned_progressed[idx] += (story.originalEstimate * uss.percentAchieved)/story.originalEstimate
                    planned_remaining[idx] += (story.originalEstimate * (1-uss.percentAchieved))/story.originalEstimate
         
        data = []
        data.append(unplanned)
        data.append(completed)
        data.append(planned_progressed)
        data.append(planned_remaining)
        data.append(not_started)
        data.append(regressed)
        data_df = pd.DataFrame(data, columns=cols)
        return data_df
    
    # Returns a dataframe of merged dataframes across all sprints and teams for a given snaphot name.
    # -snapshot_name: a string for the type of snapshot we will merge across all sprints and teams.
    # Possible choices:
    #   'planned_Start_CURRENT_SPRINT'
    #   'planned_End_CURRENT_SPRINT'
    #   'planned_Start_NEXT_SPRINT'
    #   'planned_End_NEXT_SPRINT'
    #   'backlog'
    #   'Resourcing'    
    #   'Outcome'    
    def mergeLogs(self, snapshot_name):
        log = self.log
        frames = []
        for teamId in log.keys():
            for sprint in log[teamId].keys():
                snapshot_df = log[teamId][sprint][snapshot_name]
                snapshot_df['Team Id'] = teamId
                snapshot_df['Sprint'] = sprint
                frames.append(snapshot_df)
        merged_df = pd.concat(frames)
        merged_df = merged_df.reset_index().drop(columns=['index'])
        return merged_df 

In [15]:
def runReleaseCycle(startDate, sprintDuration, numberOfSprints, modelsConfig):
# Runs the dynamics of release cycle, iterating through all the sprints. For each sprint a WorkAssignment work sheet 
# is created for each team, recording what the team aims to do during the sprint. At the end of the sprint the 
# work sheet is updated with what was actually accomplished.
#
# This function returns two objects: a detailed timecards dataframe (one row for each row in any of the sheets, across all 
# teams and all sprints), and a ReleaseLog instance
# The timecards dataframes has only scalars. The work sheet dataframe has WorkAssignment objects in one column, and an
# informative list of backlog items not yet planned.
#
# Input parameters:
# -startDate: a datetime object for when the release cycle starts
# -sprintDuration: an integer representing the number of man-days that a sprint lasts.
# -numberOfSprints: an integer, representing for how many sprints are we going to run the release. This should be set
# high enough that all work (including delays and bug fixing) is completed within these numberOfSprints, even if the 
# original estimates are for much less. The reason for needing to set it high enough is that timecard entries will stop
# after numberOfSprints.
# -modelsConfig: a ModelsConfig instance. Has the simulation models and parameters used to drive the simulated release
# cycle.
    
    timecards_dict = {'Sprint': [], 'Date': [], 'Team': [],'Developer': [], \
                      'User Story': [], 'Task Type': [], 'Task Description': [],'Time Spent': []}
    
    #Used to record how decisions are made at each sprint.
    log = ReleaseLog()
    
    sprintEndDate = startDate
    for n in range(numberOfSprints):
        sprintEndDate = addBusinessDays(sprintEndDate, sprintDuration)
        sprint = n+1
        for team in modelsConfig.globalRepo.teamsRepo.teams:
            teamId = team.teamId

            # Set context to inform the environment in which decisions and events for this cycle occur
            modelsConfig.context = ReleaseCycleContext(teamId, sprint, sprintDuration)
            
            log.snapshotSprintBacklog(modelsConfig) 

            # Do the sprint
            work = chooseWhatToDoInSprint(modelsConfig) 
            log.snapshotSprintPlan(work, modelsConfig) 

            deliverSprint(work, modelsConfig) # mutates 'work'
            inflow = inflowOfTickets(modelsConfig) #Bugs reported this sprint. Should be for stories completed on prior sprints
            updateBacklogAfterSprint(work, modelsConfig) # Does not mutate 'work' but mutates the UserStoryStatus's
            
            log.snapshotSprintOutcome(work, modelsConfig) 


            sprintOutcome = work.committedTasks()
            for index, row in sprintOutcome.iterrows():
                timecards_dict['Sprint'].append(n+1)
                timecards_dict['Date'].append(sprintEndDate)
                timecards_dict['Developer'].append(row['Owner'])
                timecards_dict['Team'].append(teamId)
                timecards_dict['User Story'].append(row['User Story Id'])
                timecards_dict['Task Type'].append(row['Task Type'])
                timecards_dict['Task Description'].append(row['Task Description'])
                timecards_dict['Time Spent'].append(row['Effort Spent']) #Even 0, record it to indicate it was in scope
            # Boundary case: if we finished all work before all sprints are through, enter 0 time spent
            if (sprintOutcome.index.size == 0):
                timecards_dict['Sprint'].append(n+1)
                timecards_dict['Date'].append(sprintEndDate)
                timecards_dict['Developer'].append(None)
                timecards_dict['Team'].append(teamId)
                timecards_dict['User Story'].append(None)
                timecards_dict['Task Type'].append(None)
                timecards_dict['Task Description'].append(None)
                timecards_dict['Time Spent'].append(0) 
                        
    return pd.DataFrame(timecards_dict), log

In [5]:
# Number of user stories worked on in a Sprint, either to implement or fix bugs on them
def countUniques(seriesGroup):
    cleanedSeriesGroup = pd.Series([x for x in seriesGroup if x != None]) # Remove nulls as they break the count of uniques
    return cleanedSeriesGroup.unique().size

def releaseBurnout(timecard):
    
    bystory             = timecard.groupby('User Story')
    ends                = bystory['Sprint'].max()
    starts              = bystory['Sprint'].min()

    impl                = timecard[timecard['Task Type'] == S_.UNFINISHED_STORIES]
    bugs                = timecard[timecard['Task Type'] == S_.DEV_TIME_BUGS]

    bysprint_all        = timecard.groupby('Sprint')
    bysprint_impl       = impl.groupby('Sprint')
    bysprint_bugs       = bugs.groupby('Sprint')
    
    counts_all          = bysprint_all['User Story'].apply(lambda x: countUniques(x)) 
    counts_impl         = bysprint_impl['User Story'].apply(lambda x: countUniques(x)) 
    counts_bugs         = bysprint_bugs['User Story'].apply(lambda x: countUniques(x)) 


    efforts_all         = bysprint_all['Time Spent'].sum()
    efforts_impl        = bysprint_impl['Time Spent'].sum()
    efforts_bugs        = bysprint_bugs['Time Spent'].sum()


    d = {'Sprint': [], 'Stories Started': [], 'Stories Active': [], 'Implementing (# stories)': [], \
         'Debugging (# stories)': [], 'Stories Completed': [], 'Cum % Completion': [], \
         'Effort': [], 'Implementation Effort': [], 'Debugging Effort': [], 'Active Developers': []}

    # Initialize
    for sprint in counts_all.index:
        d['Sprint']                                 .append(sprint)
        d['Stories Active']                         .append(counts_all[sprint])
        if sprint in counts_impl.index:
            d['Implementing (# stories)']           .append(counts_impl[sprint])
        else:
            d['Implementing (# stories)']           .append(0.0)
        
        if sprint in counts_bugs.index:
            d['Debugging (# stories)']              .append(counts_bugs[sprint])
        else:
            d['Debugging (# stories)']              .append(0.0)
        
        d['Effort']                                 .append(efforts_all[sprint])
        if sprint in efforts_impl.index:
            d['Implementation Effort']              .append(efforts_impl[sprint])
        else:
            d['Implementation Effort']              .append(0.0)
        if sprint in efforts_bugs.index:        
            d['Debugging Effort']                   .append(efforts_bugs[sprint])
        else:
            d['Debugging Effort']                   .append(0.0)
            
        for s,group in bysprint_all:
            if s==sprint: #found it
                count = group['Developer'].unique().size
                if (count == 1): #It might be that actually no developer has work left, if the 'unique' is None
                    if (None == (group['Developer'].unique())[0]):
                        count = 0
                d['Active Developers'].append(count)

        # Allocate space, we'll come back to set a value
        d['Stories Completed'].append(0)
        d['Cum % Completion'].append(0)
        d['Stories Started'].append(0) 

    for story in ends.index:
        sprint = ends[story]
        if sprint in d['Sprint']:
            idx = d['Sprint'].index(sprint)
            d['Stories Completed'][idx] += 1
        else:
            d['Sprint'].append(sprint)
            d['Stories Completed'].append(1)
            d['Stories Started'].append(0) # Allocate space, we'll come back to set a value
    for story in starts.index:
        sprint = starts[story]
        if sprint in d['Sprint']:
            idx = d['Sprint'].index(sprint)
            d['Stories Started'][idx] += 1
        else:
            d['Sprint'].append(sprint)
            d['Stories Started'].append(1)
        
        
    df = pd.DataFrame(d)
    df = df.sort_values(by='Sprint')
    
    number_of_stories_completed = df['Stories Completed'].sum()
    if number_of_stories_completed > 0:
        df['Cum % Completion'] = df['Stories Completed'].cumsum()/df['Stories Completed'].sum()
    
    return df

# Returns a dataframe, with one row per developer, stating the number of person-days they worked for each of
# the sprints of interest
#
# -sprints: a list of integers, corresponding to the sprints for which a utilization-per-developer information is sought.
def utilization(entries_df, sprints):
    fragments = []
    cols = []
    for sprint in sprints:
        df = entries_df[entries_df['Sprint'] == sprint]
        fragments.append(df.groupby('Developer')['Time Spent'].sum())
        cols.append('Sprint ' + str(sprint))
    combined_df = pd.concat(fragments, axis=1, sort=False)
    combined_df.columns = cols
    combined_df.reset_index(inplace = True)
    return combined_df

In [4]:
# Returns a dictionary. There is an entry for each user story that has been re-allocated from one developer to
# another in the course a completed release cycle, indicating all the assignment events.
# The keys in the returned dictionary are user story ids, and the value is a list of pairs 
# [sprint number, developer story was allocated to]. In other words, an assignment event.
#
# -modelsConfig: simulation configuration for the release cycle in question. It is assumed that the release
# cycle is completed prior to calling this function, so that the repositories referenced by 'modelsConfig'
# contain repos that indicate final state of user story completion after the release cycle.
def tabulateWorkStolen(modelsConfig):
    workStolen = {}
    globalRepo = modelsConfig.globalRepo
    for story in globalRepo.storiesRepo.stories:
        uss = globalRepo.teamsRepo.getUserStoryStatus(story.userStoryId)
        if len(uss.assignmentEventsLog) > 1:
            workStolen[story.userStoryId] = uss.assignmentEventsLog

    return workStolen

# Returns a Pandas dataframe corresponding to all the log's Outcome' entries in the log right before and after
# work is 'stolen' for the given 'userStoryId', i.e., an entry for the sprint where the user story is allocated to
# a different developer as well as an entry for the prior sprint showing who was the original developer from whom
# the work was stolen, and what level of incomplete work that developer had left the user story in. As a story may
# be 'stolen' multiple times in the course of a release, the dataframe may return more than two rows.
#
# -workStolen: a dictionary, like what is returned by the function 'tabulateWorkStoken': keys are user story ids, and
# value is a list of pairs [sprint number, developer story was allocated to]
def explainWorkStolen(workStolen, userStoryId, log):
    theft_sprints = []
    thefts = workStolen[userStoryId][1:] # The first time story is assigned is not a theft
    for theft in thefts:
        theft_sprints.append(theft[0])
     
    teams = list(log.log.keys())
    fragments = []
    for sprint in theft_sprints:
        for team in log.log.keys():
            if sprint > 1: #In boundary cases, a re-assignment may happen in sprint 1. Avoid keyError exceptions.
                out_previous           = log.log[team][sprint-1]   ['Outcome']
                out_previous['Sprint'] = sprint-1
                story_entries_previous = out_previous[out_previous ['User Story Id'] == userStoryId]
                fragments.append(story_entries_previous)
            
            out                    = log.log[team][sprint]     ['Outcome']
            out         ['Sprint'] = sprint
            story_entries          = out         [out          ['User Story Id'] == userStoryId]
            fragments.append(story_entries)
            
            # If this is the last allocation event, then to reflect how the user story finally gets completed
            # include all other entries for this user stories past this sprint
            if (sprint == theft_sprints[-1]): 
                last_sprint = list(log.log[team].keys())[-1]
                for s in range(sprint+1, last_sprint):
                    remaining            = log.log[team][s]    ['Outcome']
                    remaining['Sprint']  = s
                    story_remaining      = remaining[remaining ['User Story Id'] == userStoryId]
                    fragments.append(story_remaining)
            
            
    df = pd.concat(fragments)
    # Move 'Sprint' column to be the first column, for easier-to-understand display, instead of being the last column
    cols = df.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    df = df[cols]
    return df

# Returns a dataframe. Takes as input a dictionary as produced by 'tabulateWorkStolen' and returns an equivalent
# DataFrame where each row correspond to an assignment event.
def buildWorkStolen_df(workStolen):
    data_dict = {'User Story Id': [], 'Assigned on Sprint': [], 'Assigned to Developer': []}
    for story in workStolen.keys():
        allocation_events = workStolen[story]
        for event in allocation_events:
            data_dict['User Story Id'].append(story)
            data_dict['Assigned on Sprint'].append(event[0])
            data_dict['Assigned to Developer'].append(event[1])
            
    return pd.DataFrame(data_dict)

# Returns three dataframes aiming to explain how work stealing decisions were made for the given 'sprint':
# 1) Datafrae of candidate user stories to steal, based on who are the laggards
# 2) Dataframe of candidate user stories to steal, based on a balance criterion: if a developer has no user story 
#    to implement but others have multiple stories to implement, steal from the latter and give to the former.
# 3) A dataframe of what was actually stolen for the sprint. Should be a subset of the other two.
def workStealingRetrospective(sprint, log, modelsConfig):
    # Candidate work to steal for sprint due to laggards
    outcome             = log.log['Team A'][sprint-1]['Outcome']
    y                   = outcome[outcome['Delivered in Sprint'] == S_.NOT_SET]
    z                   = y[y['Most Recently Assigned in Sprint'] < sprint-2]
    laggardCandidates   = z[z['% Global Done'] <= 0.5]

    # Candidate work to steal due to balance issues
    resourcing          = log.log['Team A'][sprint-1]['Resourcing']
    steal_from          = list(resourcing[resourcing['Implementation (#)'] > 1]['Developer'])
    f                   = outcome[outcome['Owner'].isin(steal_from)]
    balanceCandidates   = f[f['% Global Done'] != 1.0]
    
    # Work actually stolen for sprint N
    workStolen          = tabulateWorkStolen(modelsConfig)
    a                   = buildWorkStolen_df(workStolen)
    stolenActuals       = a[a['Assigned on Sprint']==sprint]
    
    return laggardCandidates, balanceCandidates, stolenActuals